In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import datetime
from glob import glob
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [ ]:
data = pd.read_csv(f'../data/data/합천_댐기상종합_forTrain.csv',
                       encoding="utf-8-sig", index_col=0)
data.index = pd.to_datetime(data.index)
data.head()


In [ ]:
plt.rc('font', family='AppleGothic') 			## 이 두 줄을 
plt.rcParams['axes.unicode_minus'] = False  ## 추가해줍니다. 
plt.rcParams['axes.grid'] = False

In [ ]:
scaled_data = scaler.fit_transform(data)
data = pd.DataFrame(
    data=scaled_data, columns=df.columns, index=df.index.values)
# data.reset_index(drop=True, inplace=True)


In [ ]:
datax = data.drop(["당일유입량"], axis=1)
datay = data[['당일유입량']]

In [ ]:
import statsmodels.api as sm

datax = sm.add_constant(datax)
model = sm.OLS(datay, datax).fit()
print(model.summary())

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
 
df = data.copy()
## 전진 단계별 선택법
# variables = df.columns[:-2].tolist() ## 설명 변수 리스트
variables = datax.columns[1:].tolist()
 
# y = df['Survival_Time'] ## 반응 변수
y=datay.copy()
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = df[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = df[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 5
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()


In [ ]:
selected_variables

In [ ]:
newdf = datax[selected_variables]
vif2 = pd.DataFrame()
vif2["VIF Factor"] = [format(variance_inflation_factor(
    newdf.values, i),'f') for i in range(newdf.shape[1])]
vif2["features"] = newdf.columns
vif2.sort_values("VIF Factor")


In [ ]:
fd = newdf.drop(['저수량(현재)','저수량(예년)','이슬점온도(°C)','지면온도(°C)','저수위(예년)','증기압(hPa)','현지기압(hPa)','기온(°C)','저수량변화'], axis=1)
vif2 = pd.DataFrame()
vif2["VIF Factor"] = [format(variance_inflation_factor(
    fd.values, i),'f') for i in range(fd.shape[1])]
vif2["features"] = fd.columns
vif2.sort_values("VIF Factor")



In [ ]:
vif2['features'].values

In [ ]:
dfc = data.corr()
plt.rcParams["figure.figsize"] = (20,20)

mask = np.zeros_like(dfc, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(dfc, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
sns.pairplot(data,corner=True)

In [ ]:
data = pd.read_csv(f'../data/data/합천_댐기상종합_forTrain4_scaled.csv',
                       encoding="utf-8-sig", index_col=0)
# data.index = pd.to_datetime(data.index)
data.head()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(data.전일유입량.diff())

In [ ]:

sns.lineplot(dailyDf, x=dailyDf.index, y='전일유입량')
# sns.lineplot(dailyDf, x='시간', y='발전량(실적)', label="실적량")
plt.ylabel("(mm)")
plt.title("전체 전일유입량")

In [ ]:
# plot with 2 different axes for a dataset with different scales
# left side

fig, ax1 = plt.subplots()
fig = plt.figure(figsize=(56,9))
plt.rcParams["figure.figsize"] = (14,4)


color_1 = 'tab:blue'
ax1.set_title('실적 발전량과 계획대비 발전률')
ax1.set_ylabel('실적발전량', color=color_1)
ax1.plot(dailyDf.index, dailyDf['발전량(실적)'], color=color_1)
ax1.tick_params(axis='y', labelcolor=color_1)

# right side with different scale
ax2 = ax1.twinx() # instantiate a second axes that shares the same x-axis
color_2 = 'tab:red'
ax2.set_ylabel('계획대비 발전률', color=color_2)
ax2.plot(dailyDf.index, dailyDf['발전량(계획대비)'],  color=color_2)
ax2.tick_params(axis='y', labelcolor=color_2)


fig.tight_layout()
plt.show()

In [ ]:
sns.lineplot((dailyDf['발전량(실적)']-dailyDf['발전량(계획)']))
plt.title("전체 실적발전량-계획발전량")
plt.ylabel('(MwH)')
plt.axhline(0)

In [ ]:
dailyDf['실적-계획'] = np.where((dailyDf['발전량(실적)']-dailyDf['발전량(계획)'])>=0 , '초과' , '부족')
dailyDf['실적-계획'] = np.where((dailyDf['발전량(실적)']-dailyDf['발전량(계획)'])==0 , '적절계획' , dailyDf['실적-계획'])



In [ ]:
df2016 = dailyDf['2016-01-01 00:00:00': '2017-01-01 00:00:00']
df2017 = dailyDf['2017-01-01 00:00:00': '2018-01-01 00:00:00']
df2018 = dailyDf['2018-01-01 00:00:00': '2019-01-01 00:00:00']
df2019 = dailyDf['2019-01-01 00:00:00': '2020-01-01 00:00:00']
df2020 = dailyDf['2020-01-01 00:00:00': '2021-01-01 00:00:00']
df2021 = dailyDf['2021-01-01 00:00:00': '2022-01-01 00:00:00']



In [ ]:
plt.figure(figsize=(20,15))
plt.title('16-21 전일유입량')

plt.subplot(3,2,1)
plt.plot(df2016.index, df2016['전일유입량'])
plt.title('2016')

plt.subplot(3,2,2)
plt.plot(df2017.index, df2017['전일유입량'])
plt.title('2017')

plt.subplot(3,2,3)
plt.plot(df2018.index, df2018['전일유입량'])
plt.title('2018')

plt.subplot(3,2,4)
plt.plot(df2019.index, df2019['전일유입량'])
plt.title('2019')

plt.subplot(3,2,5)
plt.plot(df2020.index, df2020['전일유입량'])
plt.title('2020')

plt.subplot(3,2,6)
plt.plot(df2021.index, df2021['전일유입량'])
plt.title('2021')



In [ ]:
dailyDf['실적-계획'].unique()

In [ ]:
plt.pie(dailyDf['실적-계획'].value_counts(), labels=dailyDf['실적-계획'].unique(),autopct='%1.1f%%', startangle=90, explode=(0.1,0,0))
plt.title("계획발전량과 실적발전량 비교")